In [ ]:
import sys
import time
import logging
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tf_pose import common
import tf_slim as slim
from tf_pose.estimator import TfPoseEstimator
from tf_pose.networks import get_graph_path, model_wh

In [ ]:
model='cmu'
resize='480x432'
w, h = model_wh(resize)

In [ ]:
e = TfPoseEstimator(get_graph_path(model), target_size=(w, h))

In [ ]:
video_path = '/Users/andrei-macpro/Documents/Data/trial.mp4'

In [ ]:
def extract(human):
    """takes a human openpose object and returns parsed skeleton coordinates and probabilities """
    all_points = [x for x in range(18)]
    skeleton =[]
    probs = []
    kpts = []
    coords_x= []
    coords_y= []
    subscriptable = str(human).split("BodyPart:")[1:]
    for x in subscriptable:
        kpts.append(int((x.split('-')[0])))
        coords_x.append(float((str(x.split('-')[1]).split(" score=")[0][1:5]))*image.shape[1])
        coords_y.append(float((str(x.split('-')[1]).split(" score=")[0][6:11]))*image.shape[0])
        probs.append(float(str(x.split('-')[1]).split(" score=")[1]))
    
    for point in range(len(all_points)): # this ensures that undetected kpoints still have a value (nan) 
        if all_points[point] not in kpts:
            kpts.insert(all_points[point], point)
            coords_x.insert(all_points[point], "nan")
            coords_y.insert(all_points[point], "nan")
            probs.insert(all_points[point], 'nan')
    
    for k,cox, coy, prob in zip(kpts, coords_x, coords_y, probs): 
        if type(cox) == str:
            skeleton.append([cox,coy,prob])
        else:
            skeleton.append([round(cox,2),round(coy,2), prob])
    return(np.array(skeleton))

In [ ]:
def show_keypoints(image, hum, human=2, color='orange', showBG = True):
    if human == 0: human = 1
    num_hum = len(hum)
    keypoints = str(str(str(hum[human-1]).split('BodyPart:')[1:]).split('-')).split(' score=')
    keypoints_list=[]
    for i in range (len(keypoints)-1): 
        pnt = keypoints[i][-11:-1]
        pnt = tuple(map(float, pnt.split(', ')))
        keypoints_list.append(pnt)

    keypts_array = np.array(keypoints_list)
    keypts_array = keypts_array*(image.shape[1],image.shape[0])
    keypts_array = keypts_array.astype(int)
    keypts_array

    plt.figure(figsize=(10,10))
    plt.axis([0, image.shape[1], 0, image.shape[0]])  
    plt.scatter(*zip(*keypts_array), s=200, color=color, alpha=0.6)
    if showBG:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)      
    plt.imshow(image)
    ax=plt.gca() 
    ax.set_ylim(ax.get_ylim()[::-1]) 
    ax.xaxis.tick_top() 
    plt.title('Keypoints Person [{}] from {} humans detected\n'.format(human, num_hum))
    plt.grid();

    for i, txt in enumerate(keypts_array):
        ax.annotate(i, (keypts_array[i][0]-5, keypts_array[i][1]+5))
            
    return keypts_array

In [ ]:
cap = cv2.VideoCapture(video_path)
frame_no = 0
frames_list = []
while(cap.isOpened()):
    ret_val, image = cap.read()
    cv2.imshow('image',image)
    cv2.waitKey(0)
    if ret_val == False:
        break
    frames_list.append(image)
    print(frames)
    frame_no+=1
    if frame_no >10:
        break
cap.release()

In [ ]:
cv2.imshow('image',frames_list[0])
cv2.waitKey(0)

In [ ]:
humans = e.inference(frames_list[5], resize_to_default=(w > 0 and h > 0), upsample_size=4.0)
black_background = np.zeros(image.shape)
skeleton = TfPoseEstimator.draw_humans(black_background, humans, imgcopy=False)
plt.figure(figsize=(15,8))
plt.imshow(skeleton);
plt.grid();      
#plt.axis('off');

In [ ]:
keypoints = show_keypoints(frames_list[5], humans, showBG=False)

In [ ]:
keypoints

In [ ]:
humans[1]

In [ ]:
skeleton_1 = extract(humans[0])
skeleton_2 = extract(humans[1])


In [ ]:
skeleton_2

In [ ]:

showBG = True # False to show skeleton only.

cap = cv2.VideoCapture(video_path)
if cap.isOpened() is False:
    print("Error opening video stream or file")
while True:
    ret_val, image = cap.read()

    humans = e.inference(image,
                         resize_to_default=(w > 0 and h > 0),
                         upsample_size=4.0)
    if not showBG:
        image = np.zeros(image.shape)
    image = TfPoseEstimator.draw_humans(image, humans, imgcopy=False)

    cv2.imshow('tf-pose-estimation result', image)
    fps_time = time.time()
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()